In [0]:
df_bronze = spark.read.table("db_work_databricks.bronze_cc").select(
    "data",
    "descricao",
    "valor",
    "valor_original",
    "tipo_movimentacao"
)

In [0]:
import pandas as pd
from typing import Dict, List, Optional, Tuple

class ProcessamentoFinanceiro:
    def __init__(self):
        
        # [Mantido o código anterior dos mapeamentos...]
        self.mapeamento_motivos = {
            'pagto fatura master': 'Pagamento cartão de credito',
            'aplic.financ.aviso previo': 'Saida para investimento',
            'credito folha pgto.': 'Salario 25',
            'liquidacao boleto sicredi  ziani e': 'Condominio',
            'adto. salario mes': 'Salario 10',
            'liquidacao boleto  pjbank pagament': 'Aluguel',
            'resg.aplic.fin.aviso prev': 'Resgate de investimento',
            'liquidacao boleto sicredi  rede co': 'Internet',
            'pagamento pix sicredi  rede conesu': 'Internet',
            'debito convenios  rge sul-g': 'Luz',
            'debito convenios id  adm.c': 'Consorcio',
            'liquidacao boleto  pjbank': 'Aluguel',
            'pagamento pix  pjbank pagamentos s': 'Aluguel',
            'pagamento bolsa auxilio': 'Salario Estagio',
            'liquidacao boleto  lopes planos': 'Plano de Saude (Angelus)'
        }
        
        # Casos especiais baseados em valor e descrição
        self.casos_especiais_motivos = [
            {'valor': 109, 'descricao': None, 'motivo': 'Academia'},
            {'valor': 95, 'descricao': 'pagamento pix sicredi  rafael mardega', 'motivo': 'Academia'},
            {'valor': 65, 'descricao': 'pagamento pix  ana paula zalamena', 'motivo': 'Convenio Mãe (Vans)'},
            {'valor': None, 'descricao': 'pagamento pix sicredi  carla cristian', 'motivo': 'Pastel dona Carla'}
        ]
        
        self.contas_fixas = [
            'liquidacao boleto sicredi  ziani e',
            'liquidacao boleto  pjbank pagament',
            'liquidacao boleto sicredi  rede co',
            'pagamento pix sicredi  rede conesu',
            'debito convenios  rge sul-g',
            'debito convenios id  adm.c',
            'liquidacao boleto  pjbank',
            'pagamento pix  pjbank pagamentos s',
            'liquidacao boleto  lopes planos'
        ]
        
        self.investimentos = [
            'aplic.financ.aviso previo',
            'resg.aplic.fin.aviso prev'
        ]
        
        self.salarios = [
            'credito folha pgto.',
            'adto. salario mes',
            'pagamento bolsa auxilio'
        ]


    def determinar_motivo(self, row: pd.Series) -> str:
        if row['descricao'] in self.mapeamento_motivos:
            return self.mapeamento_motivos[row['descricao']]
        
        for caso in self.casos_especiais_motivos:
            if row['valor'] == caso['valor']:
                if caso['descricao'] is None or row['descricao'] == caso['descricao']:
                    return caso['motivo']
        
        return 'Outros'

    def determinar_categoria(self, row: pd.Series) -> str:
        descricao = row['descricao']
        valor = row['valor']
        
        if (descricao in self.contas_fixas or 
            valor == 109 or
            (valor == 95 and descricao == 'pagamento pix sicredi  rafael mardega') or
            (valor == 65 and descricao == 'pagamento pix  ana paula zalamena')):
            return 'Contas Fixas'
        
        if descricao in self.investimentos:
            return 'Investimento'
        
        if descricao in self.salarios:
            return 'Salario'
        
        if descricao == 'pagto fatura master':
            return 'Cartão de Crédito'
        
        return 'Outros'
    
    def determinar_meio_pagamento(self, row: pd.Series) -> str:
        """
        Determina o meio de pagamento com base na descrição e tipo de movimentação
        """
        descricao = row['descricao'].lower() if isinstance(row['descricao'], str) else ""
        tipo_movimentacao = row['tipo_movimentacao'].lower() if isinstance(row['tipo_movimentacao'], str) else ""
        
        if tipo_movimentacao == 'saída':
            if 'boleto' in descricao:
                return "Boleto (Débito Conta)"
            elif 'fatura' in descricao:
                return "Fatura Cartão de Crédito"
            elif 'debito' in descricao:
                return "Compra no Débito"
            elif 'pix' in descricao:
                return "PIX"
        return "Outros"

    def processar_dados(self, df: pd.DataFrame) -> pd.DataFrame:
        df_processado = df.copy()
        df_processado['motivo'] = df_processado.apply(self.determinar_motivo, axis=1)
        df_processado['categoria'] = df_processado.apply(self.determinar_categoria, axis=1)
        df_processado['meio_de_pagamento'] = df_processado.apply(self.determinar_meio_pagamento, axis=1)
        return df_processado




In [0]:
# Convert Spark DataFrame to pandas DataFrame
df_bronze_pd = df_bronze.toPandas()

# Process the data
processador = ProcessamentoFinanceiro()
df_processado = processador.processar_dados(df_bronze_pd)

In [0]:
display(df_processado)

In [0]:
# Convert pandas DataFrame to Spark DataFrame
df_processado_spark = spark.createDataFrame(df_processado)

# Write to Delta table
df_processado_spark.write.format("delta") \
    .mode("append") \
    .saveAsTable("db_work_databricks.prata_cc")

Com esse histórico, tenho toda base processada do inicio a 2025/08 processado e salvo no banco de dados.

- Agora para o mes 09 outro ETL pode ser feito, removendo contas que já não existem e acrescentando contas novas de POA.